In [131]:
from os import listdir
import matplotlib.pyplot as plt
import pandas as pd
import re
from fastdtw import fastdtw
from scipy.spatial import distance
import pyopenms
from operator import itemgetter
from scipy.spatial.distance import euclidean
import numpy as np

from os import listdir

### Load Ground Truth

In [132]:
def load_ground_truth_files(base_url):
    
    files = listdir(base_url)

    ground_truth = {}
    for file in files:
        #use first letter of filename as key
        ground_truth[file[0]] = pd.read_csv(base_url + file)

    return ground_truth

In [133]:
base_url = "Data//Ground Truth//"

In [136]:
ground_truth = load_ground_truth_files(base_url)

OSError: Initializing from file failed

### Test System 1 
System 1 is Matthews original code

In [86]:
#subtract UB from ground truth m/z values
def ground_truth_transform(ground_truth):
    
    #find ub row 
    mask = ground_truth['adduct'] == "Ub"
    ub_mass_charge = ground_truth['m/z'][mask].loc[0]
    
    #subtract UB mass charge off the rest
    ground_truth = ground_truth[~mask]
    ground_truth['m/z'] = ground_truth['m/z'] - ub_mass_charge
        
    return ground_truth

In [98]:
system_1_files = "Data//Binding Sites Fragment Results//"
files = listdir(system_1_files)

tolerance = 10 

In [128]:
results_dict = {}
for file in files:
    
    #load results and associated ground truth
    bonding_agent = file.split("-")[1][0].upper() #bonding agent being considered
    related_ground_truth = ground_truth[bonding_agent]
    results = pd.read_excel(system_1_files + file)
    
    #transform ground truth
    related_ground_truth = ground_truth_transform(related_ground_truth)
    
    #match the m/z values
    ground_truth_peaks = related_ground_truth['m/z'].values
    peak_predictions = results['m/z'].values
    matches_flag = np.zeros(len(ground_truth_peaks))
    
    for idx in range(0, len(ground_truth_peaks)):
        
        #extract peak and calculate upper/lower limit
        peak = ground_truth_peaks[idx]
        upper_lim = peak + tolerance
        lower_lim = peak - tolerance
        
        #match peaks
        for idx_results in range(0, len(peak_predictions)):
            pred_peak = peak_predictions[idx_results]
            if pred_peak <= upper_lim and  pred_peak >= lower_lim:
                #match found so pop from both 
                matches_flag[idx] = 1
                
                #break to ensure only one peak matches
                break
    
    results_df = pd.DataFrame({'Ground Truth m/z': ground_truth_peaks,
                  'Matches': matches_flag})
    
    
    results_dict[file] = results_df
    
    break

<ipython-input-86-09437d71d71c>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ground_truth['m/z'] = ground_truth['m/z'] - ub_mass_charge


In [125]:
#overall accuracy
total_ground_truth_peaks = 0
total_successful_matches = 0

for results in results_dict.values():
    total_ground_truth_peaks += results.shape[0]
    total_successful_matches += np.sum(results['Matches'])
    
overall_accuracy_system_1 = total_successful_matches / total_successful_matches
overall_accuracy_system_1

1.0

In [129]:
related_ground_truth = ground_truth["c"]
results = pd.read_excel(system_1_files + file)

#transform ground truth
related_ground_truth = ground_truth_transform(related_ground_truth)

,m/z,adduct,3 day intensity,8 day intensity
1,209,Ub–[Pt(NH3)],20,76
2,226,Ub–[Pt(NH3)2],83,100
3,245,Ub–[Pt(NH3)2(H2O)],1,1
4,262,Ub–[Pt(NH3)2Cl],11,1
5,436,Ub–[Pt(NH3)2+ Pt(NH3)],6,50
6,453,Ub–[Pt(NH3)2]2,1,11
7,471,Ub–[Pt(NH3)2+ Pt(NH3)2(H2O)],1,2
8,489,Ub–[Pt(NH3)2(H2O)]2 or Ub–[Pt(NH3)2Cl + Pt(NH3...,5,1


In [130]:
file

'ub_1-c_1.xlsx'